In [76]:
import pandas as pd

In [77]:
df = pd.read_csv("/Users/ibrahim/Desktop/workspace/Chat-with-csv--Using-Semantic-Search-/Data/fake_web_traffic_logs_with_cities.csv")

In [78]:
df.head()

,IP,Timestamp,HTTP_Method,URL,HTTP_Status,User_Agent,Referrer,Request_Size_Bytes,Response_Size_Bytes,Session_ID,City
0,103.40.234.168,2024-02-12 19:40:09,PUT,/checkout,403,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_11_8; ...,http://www.huang-cooper.com/,767,2186,d38d2520-0dc9-4677-af10-29329bc287b9,İzmir
1,202.139.162.161,2024-04-21 04:17:49,PUT,/home,301,Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3 like M...,http://payne.biz/,1202,2813,c1c58b10-c948-441d-bf07-f909c404ea28,Tokyo
2,200.133.227.123,2024-02-19 21:56:53,GET,/about,500,Mozilla/5.0 (iPod; U; CPU iPhone OS 3_2 like M...,http://www.gardner.biz/,959,792,55b83105-02f5-444f-b085-16e72be52014,Ankara
3,157.195.251.41,2024-06-23 13:17:34,POST,/products,403,Mozilla/5.0 (compatible; MSIE 7.0; Windows 98;...,http://alvarado-walker.com/,731,4710,8fdb7469-1e0a-4ac4-9548-2969aefd84c5,İzmir
4,142.131.0.192,2024-02-01 17:10:07,DELETE,/home,500,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_10_0 r...,http://murray-adams.com/,948,3138,4b88da64-f9e5-4cc2-84d7-8c880b340625,New York


In [79]:
print(len(df["City"].unique()))
print(df["City"].unique())

15
['İzmir' 'Tokyo' 'Ankara' 'New York' 'Edirne' 'Bursa' 'Berlin' 'Londra'
 'İstanbul' 'Paris' 'Konya' 'Diyarbakır' 'Gaziantep' 'Antalya' 'Adana']


Bu kodu yazma amacım, veri setindeki "City" sütununda kaç farklı şehir bulunduğunu ve bu şehirlerin neler olduğunu tespit etmektir. İlk satır, veri setindeki benzersiz şehirlerin sayısını belirlemek için kullanılırken, ikinci satır ise bu şehirlerin isimlerini bir liste olarak sunar.

Bu tür bir analiz, veri setinde coğrafi çeşitliliği anlamak, şehir bazında yapılan analizlerin daha doğru ve hedefli olmasını sağlamak amacıyla önemlidir. Ayrıca, bu bilgilere dayanarak, modelin veya analizin odaklanacağı spesifik şehirleri belirlemek mümkün olur.


## Gereksiz Sütunları Temizleyelim

In [80]:
df.columns

Index(['IP', 'Timestamp', 'HTTP_Method', 'URL', 'HTTP_Status', 'User_Agent',
       'Referrer', 'Request_Size_Bytes', 'Response_Size_Bytes', 'Session_ID',
       'City'],
      dtype='object')

In [81]:
df.drop(["IP","Timestamp", "Referrer", "Session_ID","User_Agent"], axis = 1, inplace=True)

Bu veri setinden "IP", "Timestamp", "Referrer", "Session_ID" ve "User_Agent" sütunlarını çıkardım çünkü LLM (Büyük Dil Modeli) kullanarak veri ile bir Retrieval-Augmented Generation (RAG) yaklaşımıyla kullanıcı sorgularına yanıt üretmeyi hedefliyorum. Bu sütunlar, modelin yanıt üretim sürecinde gereksiz veya aşırı bilgi yükü oluşturabilecek meta verilerdir. Modelin asıl odaklanması gereken içerik ve kullanıcı sorgularıyla daha alakalı sütunlardır; dolayısıyla bu verileri çıkararak, modelin daha verimli ve odaklı çalışmasını sağlamak istiyorum.


In [82]:
df.head()

,HTTP_Method,URL,HTTP_Status,Request_Size_Bytes,Response_Size_Bytes,City
0,PUT,/checkout,403,767,2186,İzmir
1,PUT,/home,301,1202,2813,Tokyo
2,GET,/about,500,959,792,Ankara
3,POST,/products,403,731,4710,İzmir
4,DELETE,/home,500,948,3138,New York


In [83]:
# Sütun isimlerini Türkçe olarak yeniden adlandıralım
df.columns = [
    'HTTP Yöntemi', 'URL', 'Durum Kodu', 'İstek Boyutu (Bayt)', 
    'Yanıt Boyutu (Bayt)', 'Konum'
]

df.head()


,HTTP Yöntemi,URL,Durum Kodu,İstek Boyutu (Bayt),Yanıt Boyutu (Bayt),Konum
0,PUT,/checkout,403,767,2186,İzmir
1,PUT,/home,301,1202,2813,Tokyo
2,GET,/about,500,959,792,Ankara
3,POST,/products,403,731,4710,İzmir
4,DELETE,/home,500,948,3138,New York


In [84]:
#Yanıt ve istek boyutlarının da bir soru cevap sisteminde bulunmasının gereği olmadığından onları da çıkartma kararı alalım.
df.drop(["İstek Boyutu (Bayt)",	"Yanıt Boyutu (Bayt)"], axis = 1, inplace=True)

In [85]:
df

,HTTP Yöntemi,URL,Durum Kodu,Konum
0,PUT,/checkout,403,İzmir
1,PUT,/home,301,Tokyo
2,GET,/about,500,Ankara
3,POST,/products,403,İzmir
4,DELETE,/home,500,New York
...,...,...,...,...
995,PUT,/products,404,Edirne
996,DELETE,/home,301,Adana
997,GET,/cart,200,Paris
998,GET,/services,500,Berlin


## Sistemimizde kullanmak üzere temizlediğimiz veri setini kayededelim

In [86]:
df.to_csv('data.csv', index=False)

In [87]:
#Boş değer var mı yok mu gelen olarak veriyi analiz edelim
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HTTP Yöntemi  1000 non-null   object
 1   URL           1000 non-null   object
 2   Durum Kodu    1000 non-null   int64 
 3   Konum         1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [89]:
#Web sitesinden alınan ve gönderilen veri boyutlarını analiz edelim.
df_for_t_s = pd.read_csv("/Users/ibrahim/Desktop/workspace/Chat-with-csv--Using-Semantic-Search-/Data/fake_web_traffic_logs_with_cities.csv")
Bayt_analysis = df_for_t_s[["Response_Size_Bytes", "Request_Size_Bytes"]]

In [90]:
Bayt_analysis

,Response_Size_Bytes,Request_Size_Bytes
0,2186,767
1,2813,1202
2,792,959
3,4710,731
4,3138,948
...,...,...
995,4769,1595
996,498,1777
997,4773,630
998,393,1604


In [91]:
Bayt_analysis.describe().T

,count,mean,std,min,25%,50%,75%,max
Response_Size_Bytes,1000.0,2612.813,1390.108758,202.0,1426.0,2686.0,3814.5,5000.0
Request_Size_Bytes,1000.0,1053.512,542.659445,101.0,586.0,1054.0,1514.5,1999.0


Bu analize bakıldığında, "Response_Size_Bytes" değişkeni ortalama 2612,81 bayt ile oldukça geniş bir dağılım gösteriyor, çünkü standart sapma 1390,11 bayt gibi yüksek bir değere sahip. Bu, yanıt boyutlarının önemli ölçüde değişkenlik gösterdiğini ve bazı yanıtların oldukça büyük veya küçük olabileceğini düşündürmektedir. Yanıt boyutlarının medyanı (2686 bayt), ortalamaya oldukça yakın olup, verinin sağa çarpık olmadığına işaret ediyor. En küçük yanıt boyutu 202 bayt, en büyük ise 5000 bayt olarak ölçülmüştür. Diğer taraftan, "Request_Size_Bytes" değişkeni ortalama 1053,51 bayt ile daha küçük boyutlarda olup, standart sapması 542,66 bayt ile daha düşük bir yayılım gösteriyor. Bu, istek boyutlarının yanıt boyutlarına kıyasla daha tutarlı olduğunu gösterir. İstek boyutlarının medyanı (1054 bayt) ortalamaya oldukça yakındır, bu da simetrik bir dağılımı işaret eder. En küçük istek boyutu 101 bayt, en büyük ise 1999 bayt olarak belirlenmiştir. Genel olarak, yanıt boyutları istek boyutlarına kıyasla daha geniş bir aralıkta dağılım gösterirken, istek boyutları daha tutarlı ve dar bir aralıkta seyretmektedir.

In [92]:
df

,HTTP Yöntemi,URL,Durum Kodu,Konum
0,PUT,/checkout,403,İzmir
1,PUT,/home,301,Tokyo
2,GET,/about,500,Ankara
3,POST,/products,403,İzmir
4,DELETE,/home,500,New York
...,...,...,...,...
995,PUT,/products,404,Edirne
996,DELETE,/home,301,Adana
997,GET,/cart,200,Paris
998,GET,/services,500,Berlin


## HTTP Yöntemine Göre Analiz

In [93]:
method_distribution = df["HTTP Yöntemi"].value_counts()
print(method_distribution)

HTTP Yöntemi
DELETE    260
GET       255
PUT       248
POST      237
Name: count, dtype: int64


Bu kod, veri setindeki HTTP yöntemlerinin (GET, POST, PUT, DELETE) ne sıklıkla kullanıldığını gösterir. Bu sayede en çok hangi HTTP yöntemlerinin kullanıldığını anlayabiliriz.

## Şehir Bazlı Analiz

In [94]:
city_distribution = df["Konum"].value_counts()
print(city_distribution)


Konum
Adana         78
Ankara        76
İstanbul      72
Paris         69
Konya         69
New York      68
Bursa         68
Diyarbakır    67
Tokyo         65
Edirne        63
Londra        63
İzmir         62
Berlin        61
Antalya       61
Gaziantep     58
Name: count, dtype: int64


Bu kod, hangi şehirlerden daha fazla istek geldiğini analiz eder. Bu sayede, coğrafi olarak hangi bölgelerin daha aktif olduğunu görebiliriz.

## URL Performansı

In [95]:
url_status_distribution = df.groupby("URL")["Durum Kodu"].value_counts().unstack()
print(url_status_distribution)

Durum Kodu  200  301  403  404  500
URL                                
/about       15   26   16   15   25
/blog        17   12   16   15   23
/cart        10   26   14   12   15
/checkout    20   18   13   16   16
/contact     13   16   17   18   19
/home        20   20   10   17   17
/login       12   14    9   18   28
/products    17   19   17   16   13
/profile     16    9   22   16   13
/search      17   19   19   17   27
/services    12   12   11   17   22
/signup      20   18   10   16   17


Bu yapı, farklı URL'lerin ne tür HTTP durum kodları döndürdüğünü gösterir. Bu, hangi URL'lerin başarılı olduğunu, hangilerinin ise hata verdiğini anlamak için kullanılır.

## Basit Bir Zaman Serisi Analizi

Bu blok, zaman serisi analizi yaparak belirli zaman dilimlerinde istek yoğunluğunu incelemeye olanak tanır. Bu sayede, sistemin hangi zaman dilimlerinde daha fazla yük altında olduğunu anlayabiliriz.

In [96]:
df = pd.read_csv('/Users/ibrahim/Desktop/workspace/Chat-with-csv--Using-Semantic-Search-/Data/fake_web_traffic_logs_with_cities.csv')

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Date'] = df['Timestamp'].dt.date

requests_by_date = df.groupby('Date').size()

max_requests_date = requests_by_date.idxmax()
max_requests_count = requests_by_date.max()

min_requests_date = requests_by_date.idxmin()
min_requests_count = requests_by_date.min()

print(f"En fazla istek alan tarih: {max_requests_date}, İstek Sayısı:{max_requests_count}")
print(f"En az istek alan tarih: {min_requests_date}, İstek Sayısı:{min_requests_count}")

En fazla istek alan tarih: 2024-07-21, İstek Sayısı:10
En az istek alan tarih: 2024-01-15, İstek Sayısı:1
